<a href="https://colab.research.google.com/github/Minh-A/NEW_Algorithm2/blob/main/Fundamentals%20of%20the%20GAN%20Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 바이낸스 비트코인

## 데이터 전처리

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import matplotlib.pyplot as plt

# 성능지표
from sklearn.metrics import confusion_matrix, classification_report
import sklearn.metrics as mt

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/비트코인/data/바이낸스_btc_15m.csv', encoding='cp949')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31931 entries, 0 to 31930
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  31931 non-null  object 
 1   open      31931 non-null  float64
 2   high      31931 non-null  float64
 3   low       31931 non-null  float64
 4   close     31931 non-null  float64
 5   volume    31931 non-null  float64
 6   opup      31931 non-null  float64
 7   opdown    31931 non-null  float64
 8   clup      31931 non-null  float64
 9   cldown    31931 non-null  float64
 10  updown    31931 non-null  float64
dtypes: float64(10), object(1)
memory usage: 2.7+ MB


In [5]:
data.head()

,datetime,open,high,low,close,volume,opup,opdown,clup,cldown,updown
0,2021-01-01 0:00,28923.63,29017.50,28690.17,28752.80,840.077569,-93.87,233.46,-264.70,62.63,327.33
1,2021-01-01 0:15,28752.80,28875.55,28720.91,28836.63,480.611179,-122.75,31.89,-38.92,115.72,154.64
2,2021-01-01 0:30,28836.63,28943.87,28836.62,28930.11,471.134708,-107.24,0.01,-13.76,93.49,107.25
3,2021-01-01 0:45,28930.11,29031.34,28889.99,28995.13,519.987989,-101.23,40.12,-36.21,105.14,141.35
4,2021-01-01 1:00,28995.13,29385.00,28960.35,29382.59,1735.839005,-389.87,34.78,-2.41,422.24,424.65


선형 보간법

In [ ]:
# 선형 보간법으로 결측치를 없앰
# data1 = data.interpolate(method="linear")

In [ ]:
# data1.head(3)

데이터 셋 분류

In [ ]:
# 훈련데이터 셋과 테스트데이터 셋을 분류
# train = data.iloc[:-7,:]
# test = data.iloc[-7:,:]

In [ ]:
# 각 데이터 셋에서 Feature와 label을 분류
# train_feature = train.iloc[:,1:]
# train_label = train.iloc[:,1:4]

# test_feature = test.iloc[:,1:]
# test_label = test.iloc[:,1:4]

In [ ]:
# train_feature.head(3), test_label.head(3)

원핫인코딩 처리

In [ ]:
# label은 원핫인코딩 처리
# train_label1 = pd.get_dummies(train_label)
# test_label1 = pd.get_dummies(test_label)

In [ ]:
# test_label1.head(5)

데이터 표준화를 사용

In [ ]:
# # 데이터 표준화 (StandardScaler)
# from sklearn.preprocessing import StandardScaler

# # 훈련 데이터셋 feature
# scaler = StandardScaler()

# train_feature = scaler.fit_transform(train_feature) 
# train_feature1 = pd.DataFrame(train_feature)

# # 테스트 데이터셋 feature
# scaler = StandardScaler()

# test_feature = scaler.fit_transform(test_feature)
# test_feature1 = pd.DataFrame(test_feature)

# train_feature1.head(3), test_feature1.head(3)

여기서부터 시작

In [6]:
# 데이터 표준화 (StandardScaler)
from sklearn.preprocessing import MinMaxScaler

# 훈련 데이터셋 feature
scaler = MinMaxScaler()

data1 = scaler.fit_transform(np.array(data)[:,1:]) 

In [7]:
data1.shape, " ",data1[:2]

((31931, 10),
 ' ',
 array([[0.00427437, 0.00353774, 0.01388278, 0.        , 0.04983096,
         0.97655085, 0.04267789, 0.92870495, 0.0160325 , 0.05390304],
        [0.        , 0.        , 0.01464461, 0.00209753, 0.02850846,
         0.96933649, 0.00582968, 0.98951718, 0.02962288, 0.02546533]]))

In [8]:
data1.min(), data1.max() 

(0.0, 1.0)

슬라이딩윈도우

In [9]:
# (배치 사이즈, 날짜, 속성)으로 데이터 셋을 생성
# 한칸씩 미는 식으로 구성됨

# def make_dataset(data, window_size):
#     feature_list = []
#     # label_list = []
#     for i in range(len(data) - window_size):
#         feature_list.append(np.array(data.iloc[i:i+window_size]))
#         # label_list.append(np.array(label.iloc[i+window_size]))
#     return np.array(feature_list)  # , np.array(label_list)

In [10]:
# 윈도우 사이즈만큼 단일 구성함

def make_dataset(data, window_size):
    feature_list = []
    f_num = len(data) % window_size
    print(f_num)

    feature_list.append(data.iloc[ f_num : f_num + window_size])

    for i in range(1, len(data) // window_size ):
        i *= window_size
        i += f_num
        feature_list.append(data.iloc[i:i+window_size])
    return np.array(feature_list)

In [12]:
# 데이터 셋을 슬라이딩 윈도우 형식으로 만들기
# 15분씩 1시간은 4번 하루는 96

# 훈련데이터 셋
train_feature = make_dataset(pd.DataFrame(data1), 8)

x_train = train_feature

x_train.shape


3


(3991, 8, 10)

In [13]:
x_test = x_train[3989]
x_test.shape
pd.DataFrame(x_test)

,0,1,2,3,4,5,6,7,8,9
0,0.715733,0.711251,0.721282,0.714784,0.012458,0.985948,0.022692,0.974636,0.022069,0.029704
1,0.714784,0.709897,0.719476,0.711141,0.017474,0.990045,0.029079,0.950050,0.003448,0.032757
2,0.711141,0.706376,0.709790,0.703099,0.046451,0.988966,0.073914,0.901531,0.021224,0.073857
3,0.703099,0.703809,0.711689,0.709104,0.017185,0.934406,0.001146,0.993921,0.063045,0.044273
4,0.709104,0.707161,0.715092,0.712824,0.016361,0.960758,0.019915,0.997735,0.065948,0.043808
5,0.712824,0.707842,0.715466,0.710659,0.017151,0.991069,0.044334,0.967065,0.039931,0.045824
6,0.710659,0.706191,0.716729,0.709570,0.010628,0.986003,0.019206,0.973192,0.015759,0.026527
7,0.709571,0.706710,0.717244,0.711908,0.010936,0.969944,0.007449,0.992760,0.034351,0.026524


## GAN 알고리즘

* https://teddylee777.github.io/tensorflow/vanilla-gan

In [ ]:
# from tensorflow.keras.datasets import mnist
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# x_train.min()

In [ ]:
# x_train = x_train / 127.5 - 1
# x_test = x_test / 127.5 - 1

# print(x_train.min(), x_train.max())

# x_train = x_train.reshape(-1, 784)
# x_train.shape

-1.0 1.0


(60000, 784)

In [ ]:
# x_train = data1
# x_train.shape

(31931, 10)

In [14]:
from tensorflow.keras.layers import Dense, LeakyReLU, Dropout, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
import numpy as np
import matplotlib.pyplot as plt

In [15]:
# gan에 입력되는 noise에 대한 dimension
NOISE_DIM = 10

# adam optimizer 정의, learning_rate = 0.0002, beta_1로 줍니다.
# Vanilla Gan과 DCGAN에서 이렇게 셋팅을 해주는데
# 이렇게 해줘야 훨씬 학습을 잘합니다.
adam = Adam(lr=0.0002, beta_1=0.5)

In [16]:
generator = Sequential([
    Dense(64, input_dim=NOISE_DIM), 
    LeakyReLU(0.2), 
    Dense(128), 
    LeakyReLU(0.2), 
    Dense(256), 
    LeakyReLU(0.2), 
    Dense(10, activation='tanh'),
])

In [17]:
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                704       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                2

In [18]:
discriminator = Sequential([
    Dense(256, input_shape=(10,), kernel_initializer=RandomNormal(stddev=0.02)),
    LeakyReLU(0.2), 
    Dropout(0.1), 
    Dense(128),
    LeakyReLU(0.2), 
    Dropout(0.1), 
    Dense(64),
    LeakyReLU(0.2), 
    Dropout(0.1), 
    Dense(1, activation='sigmoid')
])

In [19]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               2816      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 128)               0         
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 64)               

In [20]:
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

In [21]:
# discriminator는 학습을 하지 않도록 하며, Gan 모델에서는 generator만 학습하도록 합니다.
discriminator.trainable = False
gan_input = Input(shape=(NOISE_DIM,))
x = generator(inputs=gan_input)
output = discriminator(x)

In [22]:
gan = Model(gan_input, output)
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 sequential (Sequential)     (None, 10)                44618     
                                                                 
 sequential_1 (Sequential)   (None, 1)                 44033     
                                                                 
Total params: 88,651
Trainable params: 44,618
Non-trainable params: 44,033
_________________________________________________________________


In [23]:
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
# def get_batches(data, batch_size):
#     batches = []
#     for i in range(int(data.shape[0] // batch_size)):
#         batch = data[i * batch_size: (i + 1) * batch_size]
#         batches.append(batch)
#     return np.asarray(batches)

In [ ]:
x_train.shape

(3991, 8, 10)

In [ ]:
x_test.shape

(8, 10)

In [ ]:
# get_batches(x_train,128).shape

In [24]:
from tabulate import tabulate  # 데이터프레임 이쁘게 인쇄

def visualize_training(epoch, d_losses, g_losses):
    print('\n\n')

    # 오차에 대한 시각화
    plt.figure(figsize=(8, 4))
    plt.plot(d_losses, label='Discriminator Loss')
    plt.plot(g_losses, label='Generatror Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    print('epoch: {}, Discriminator Loss: {}, Generator Loss: {}'.format(epoch, np.asarray(d_losses).mean(), np.asarray(g_losses).mean()))
    
    #샘플 데이터 생성 후 시각화
    # noise = np.random.normal(0, 1, size=(10, NOISE_DIM))
    noise = x_test
    generated_num = generator.predict(noise)  # 생성자가 만든 데이터
    # print("\n 생성된 숫자를 표기 \n",generated_num)

    print("\n 진짜 숫자 \n", tabulate(pd.DataFrame(scaler.inverse_transform(x_train[3990]), 
                                                    columns=['open', 'high', 'low', 'close', 'volume','opup',	'opdown',	'clup',	'cldown',	'updown']), 
                                                headers='keys', tablefmt='psql'))

    print("\n 생성된 숫자의 변환 \n", tabulate(pd.DataFrame(scaler.inverse_transform(generated_num), 
                                                    columns=['open', 'high', 'low', 'close', 'volume','opup',	'opdown',	'clup',	'cldown',	'updown']), 
                                                headers='keys', tablefmt='psql'))


In [ ]:
# def visualize_training(epoch, d_losses, g_losses):
#     # 오차에 대한 시각화
#     plt.figure(figsize=(8, 4))
#     plt.plot(d_losses, label='Discriminator Loss')
#     plt.plot(g_losses, label='Generatror Loss')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()

#     print('epoch: {}, Discriminator Loss: {}, Generator Loss: {}'.format(epoch, np.asarray(d_losses).mean(), np.asarray(g_losses).mean()))
    
#     #샘플 데이터 생성 후 시각화
#     noise = np.random.normal(0, 1, size=(24, NOISE_DIM))
#     generated_images = generator.predict(noise)
#     generated_images = generated_images.reshape(-1, 28, 28)
    
#     plt.figure(figsize=(8, 4))
#     for i in range(generated_images.shape[0]):
#         plt.subplot(4, 6, i+1)
#         plt.imshow(generated_images[i], interpolation='nearest', cmap='gray')
#         plt.axis('off')
#     plt.tight_layout()
#     plt.show()

In [29]:
BATCH_SIZE = 8
EPOCHS= 5

In [26]:
# get_batches(x_train, BATCH_SIZE).shape

In [27]:
x_train.shape

(3991, 8, 10)

In [ ]:
x_test.shape

(8, 10)

In [ ]:
gan.train_on_batch(noise, y_gan)

0.7779050469398499

In [ ]:
y_gan

array([1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
x_train[-1].reshape(1,8,10).shape

(1, 8, 10)

In [ ]:
# discriminator와 gan 모델의 loss 측정을 위한 list 입니다.
# 여기서 GAN이란 가짜 사진을 받은 판별자가 TRUE, FALSE에 대한 정도를 말하는 모델(sigmoid로)
d_losses = []
g_losses = [] 

for epoch in range(1, EPOCHS + 1):
    # 각 배치별 학습
    # for real_num in get_batches(x_train, BATCH_SIZE):
    for real_num in x_train:  
        # 랜덤 노이즈 생성
        # input_noise = np.random.uniform(0, 1, size=[BATCH_SIZE, NOISE_DIM])
        input_noise = x_test  # 노이즈가 아닌 실제 값으로 예측 시도
        
        # 가짜 이미지 데이터 생성
        generated_num = generator.predict(input_noise)  # 생성자 예측
        
        # Gan에 학습할 X 데이터 정의
        x_dis = np.concatenate([real_num, generated_num])

        # print(x_dis.shape)
        # print(real_num.shape)
        # print(generated_num.shape)
        
        # Gan에 학습할 Y 데이터 정의
        y_dis = np.zeros(2 * BATCH_SIZE)
        y_dis[:BATCH_SIZE] = 0.9    # 뭐랄까 Weight값을 부여하는 거 같음(진짜에 가중치를 부여)

        # print("/n", y_dis, y_dis.shape)
        
        # Discriminator 훈련
        discriminator.trainable = True
        d_loss = discriminator.train_on_batch(x_dis, y_dis)  # 판별자 훈련, train_on_batch는 하나의 데이터 배치에 대해 경사 업데이트를 1회 실시
        
        # Gan 훈련
        # noise = np.random.uniform(0, 1, size=[BATCH_SIZE, NOISE_DIM])
        noise = x_test
        y_gan = np.ones(BATCH_SIZE)  # 1을 부여함
        
        # Discriminator의 판별 학습을 방지합니다
        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, y_gan)  # GAN모델을 훈련, 판별자 훈련을 방지했기에 생성자가 훈련된다
        
    d_losses.append(d_loss)
    g_losses.append(g_loss)
    
    if epoch == 1 or epoch % 10 == 0:
        visualize_training(epoch, d_losses, g_losses)

1. 가짜 데이터(노이즈)를 가지고 생성자가 예측을 시도함
2. 판별자는 생성자가 예측한 것을 모두 아닌 것(0)으로 보고 for문을 통한 데이터만을 진짜라고(0.9) 보며 학습함 ==> # Discriminator 훈련
3. 판별자가 '생성자가 만든 데이터'를 진짜라고 판별하게(1) 하기위해 생성자가 해당 프로세스(Gan 모델)를 거치며 학습함 ==> # Gan 훈련

In [ ]:
  y_dis = np.zeros(2 * BATCH_SIZE)
y_dis[:BATCH_SIZE] = 0.9
y_dis

array([0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.

In [ ]:
noise = np.random.uniform(-1, 1, size=[BATCH_SIZE, NOISE_DIM])
y_gan = np.ones(BATCH_SIZE)
y_gan

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.])